In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import numpy as np
import tensorflow as tf
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import *
from keras.layers import *
import matplotlib.pyplot as plt
import os

In [ ]:
init = RandomNormal(mean=0.0, stddev=0.02)

### **Tạo mạng Discriminator (Mạng phân loại thật giả)**

In [ ]:
def discriminator(image_shape = (32, 32, 3)):
  inputs = Input(shape = image_shape)
  x = Conv2D(64, kernel_size = (3, 3), strides = 1, padding = "same", kernel_initializer = init)(inputs)
  x = BatchNormalization()(x)
  x = LeakyReLU(0.2)(x)

  x = Conv2D(128, kernel_size = (3, 3), strides = 2, padding = "same", kernel_initializer = init)(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(0.2)(x)

  x = Conv2D(128, kernel_size = (3, 3), strides = 2, padding = "same", kernel_initializer = init)(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(0.2)(x)

  x = Conv2D(256, kernel_size = (3, 3), strides = 2, padding = "same", kernel_initializer = init)(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(0.2)(x)

  x = Flatten()(x)
  outputs = Dense(1, activation = "sigmoid", kernel_initializer = init)(x)

  model = Model(inputs = inputs, outputs = outputs)
  model.compile(loss = "binary_crossentropy", optimizer = Adam(lr=0.0002, beta_1=0.5), metrics = ["accuracy"])

  return model

### **Tạo mạng Generator (Mạng sinh ảnh)**

In [ ]:
def generator(latent_dim):
  
  inputs = Input(shape = (latent_dim, ))

  n_nodes = 256*4*4

  x = Dense(n_nodes, kernel_initializer = init)(inputs)
  x = LeakyReLU(0.2)(x)

  x = Reshape((4, 4, 256))(x)

  x = Conv2DTranspose(256, kernel_size = (4, 4), strides = 2, padding = "same", kernel_initializer = init)(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(0.2)(x)

  x = Conv2DTranspose(128, kernel_size = (4, 4), strides = 2, padding = "same", kernel_initializer = init)(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(0.2)(x)

  x = Conv2DTranspose(64, kernel_size = (4, 4), strides = 2, padding = "same", kernel_initializer = init)(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(0.2)(x)

  outputs = Conv2D(3, kernel_size = (3, 3), strides = 1, padding = "same", activation = "tanh", kernel_initializer = init)(x)
  model = Model(inputs = inputs, outputs = outputs)
  model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])
  return model

### **Kết hợp tạo ra mạng GAN**

In [ ]:
def simple_gan(g_model, d_model):
  d_model.trainable = False
  
  model = Sequential()

  model.add(g_model)
  
  model.add(d_model)
 
  model.compile(loss= "binary_crossentropy" , optimizer = Adam(lr=0.0002, beta_1=0.5))
  return model

### **Load dữ liệu**

In [ ]:
def smooth_true_labels(y):
  return y - 0.3 + (np.random.random(y.shape) * 0.5)

def load_real_samples():
  (trainX, _), (_, _) = load_data()
  X = trainX.astype("float32")
  X = (X - 127.5) / 127.5
  return X

def generate_real_samples(dataset, n_samples):
  ix = randint(0, dataset.shape[0], n_samples)
  X = dataset[ix]
  y = np.ones((n_samples, 1))
  return X, y

def generate_latent_points(latent_dim, n_samples):
  x_input = randn(latent_dim * n_samples)
  
  x_input = x_input.reshape(n_samples, latent_dim)
  return x_input

def generate_fake_samples(g_model, latent_dim, n_samples):
  x_input = generate_latent_points(latent_dim, n_samples)
  X = g_model.predict(x_input)
  y = np.zeros((n_samples, 1))
  return X, y

In [ ]:
model_save_path = "/content/gdrive/MyDrive/Teaching Assistant Saturday/My code/week 9/GAN_MNIST_model"

In [ ]:
import cv2
def save_plot(examples, epoch, n=10):
  examples = (examples + 1)*0.5
  # Vẽ
  for i in range(n * n):
    # Add subplot
    plt.subplot(n, n, i + 1)
    # Bỏ trục
    plt.axis("off") 
    # plot raw pixel data
    plt.imshow(examples[i])
    # Lưu ảnh
  filename = 'Image/generated_plot_e%03d.png' % (epoch+1)
  if epoch % 10 == 0:
    plt.savefig(os.path.join(model_save_path, filename))
    plt.show()  
  plt.close()

In [ ]:
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
  # Chuẩn bị real
  X_real, y_real = generate_real_samples(dataset, n_samples)
  # Phân biệt real
  _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
  # Tạo fake
  x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
  # Phân biệt fake
  _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
  # In ra acc fake và real
  print("Epoch %d: accuracy real: %.0f%%, fake: %.0f%%" % (epoch + 1, acc_real*100, acc_fake*100))
  # Lưu ảnh số
  save_plot(x_fake, epoch)
  # Lưu file
  filename = "Model/generator_model_%03d_real%.3f_fake%.3f.h5" % (epoch + 1, acc_real, acc_fake)
  g_model.save(os.path.join(model_save_path, filename))

In [ ]:
def train(gan_model, dataset, latent_dim, n_epochs=200, n_batch=256):
  bat_per_epo = int(dataset.shape[0] / n_batch)
  half_batch = int(n_batch / 2)

  for i in range(n_epochs):
    for j in range(bat_per_epo):
      # Phase 1: Huấn luyện mạng phân biệt
      gan_model.layers[0].trainable = False
      gan_model.layers[1].trainable = True

      X_real, y_real = generate_real_samples(dataset, half_batch)
      d_loss1, _ = gan_model.layers[1].train_on_batch(X_real, y_real)
      X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      d_loss2, _ = gan_model.layers[1].train_on_batch(X_fake, y_fake)

      # Phase 2: Huấn luyện mạng sinh
      gan_model.layers[0].trainable = True
      gan_model.layers[1].trainable = False

      X_gan = generate_latent_points(latent_dim, n_batch)
      y_gan = np.ones((n_batch, 1))
      g_loss = gan_model.layers[0].train_on_batch(X_gan, y_gan)
      if (j) % 10 == 0:
        print("Epoch %d, %d/%d, d1=%.3f, d2=%.3f g=%.3f" %(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))

    summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [ ]:

latent_dim = 100
d_model = discriminator()
g_model = generator(latent_dim) 
gan_model = simple_gan(g_model, d_model)
dataset = load_real_samples()

x = train(gan_model, dataset, latent_dim)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1, 1/195, d1=0.652, d2=1.439 g=0.694


KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
!zip -r "/content/gdrive/MyDrive/Teaching Assistant Saturday/My code/week 9/gan.zip" "/content/GAN"

  adding: content/GAN/ (stored 0%)
  adding: content/GAN/.ipynb_checkpoints/ (stored 0%)
  adding: content/GAN/generator_model_076_real0.400_fake0.990.h5 (deflated 9%)
  adding: content/GAN/generator_model_084_real0.780_fake0.890.h5 (deflated 9%)
  adding: content/GAN/generator_model_063_real0.970_fake0.300.h5 (deflated 9%)
  adding: content/GAN/generated_plot_e098.png (deflated 11%)
  adding: content/GAN/generator_model_097_real0.230_fake1.000.h5 (deflated 9%)
  adding: content/GAN/generator_model_068_real0.830_fake0.940.h5 (deflated 9%)
  adding: content/GAN/generated_plot_e096.png (deflated 11%)
  adding: content/GAN/generated_plot_e097.png (deflated 11%)
  adding: content/GAN/generator_model_072_real0.200_fake1.000.h5 (deflated 9%)
  adding: content/GAN/generated_plot_e090.png (deflated 11%)
  adding: content/GAN/generator_model_070_real0.290_fake1.000.h5 (deflated 9%)
  adding: content/GAN/generated_plot_e069.png (deflated 11%)
  adding: content/GAN/generated_plot_e079.png (deflat